# Extrator de informações genérico

In [19]:
import pandas as pd
import os
from bs4 import BeautifulSoup
import re

## Criando lista de páginas

In [20]:
soups = []
files = []
for dirpath,_,filenames in os.walk(os.path.abspath('..\\Pages\\Positives\\')):
    for f in filenames:
        file = os.path.abspath(os.path.join(dirpath, f))
        files.append(file)
        soups.append(BeautifulSoup(
        open(file, encoding='utf-8'), 'html.parser'))
print(len(files))
len(soups)

81


81

## Extraindo nomes dos psicologos

Antes de começarmos vamos examinar rapidamente os campos h1's das páginas, uma vez que imaginasse que o nome do psicólogo deva ser encontrado neste campo

In [21]:
def clean_text(text):
    return text.replace('\n', '').replace('\t', '').strip();

for index, soup in enumerate(soups):
    h1s = soup.find_all('h1')
    if(len(h1s) == 1):
        print(str(index) + ' - ' + str(len(h1s)) + ' -- ' + clean_text(h1s[0].text))
    elif(len(h1s) == 2):
        print(str(index) + ' - ' + str(len(h1s)) + ' - ' + str(h1s[0].parent.name) +
              ' - ' + clean_text(h1s[0].text) + ' -- ' + str(h1s[1].parent.name) + ' - '+ clean_text(h1s[1].text))
    else:
        print( str(index) + ' - Empty result')



0 - Empty result
1 - Empty result
2 - Empty result
3 - Empty result
4 - Empty result
5 - Empty result
6 - Empty result
7 - Empty result
8 - Empty result
9 - Empty result
10 - 2 - div - Sérgio Alexandre Alves Fernandes -- div - Sérgio Alexandre Alves Fernandes
11 - 2 - div - Luiz Gustavo Canuto -- div - Luiz Gustavo Canuto
12 - 2 - div - Igor Costa -- div - Igor Costa
13 - 2 - div - Bruno Schievenin Madi -- div - Bruno Schievenin Madi
14 - 2 - div - Sandro Azevêdo -- div - Sandro Azevêdo
15 - 2 - div - Wilza M Brosig -- div - Wilza M Brosig
16 - 2 - div - Dra. Luciana Dias Nascimento -- div - Dra. Luciana Dias Nascimento
17 - 2 - div - Fernanda Simionato Costa -- div - Fernanda Simionato Costa
18 - 2 - div - Rodrigo Noia Mattos Montan -- div - Rodrigo Noia Mattos Montan
19 - 2 - div - Kassia Arouca cardoso -- div - Kassia Arouca cardoso
20 - 2 - div - Dr. Caio Fasoli Rebouças -- div - Dr. Caio Fasoli Rebouças
21 - 1 -- ADRIANA PANZAN
22 - 1 -- CONSULTORIO DE PSICOLOGIA SORAIA S GRANADO 

Analisando a coleta acima é possível perceber que existem poucos padrões diferentes.

Primeiro podemos identificar que as páginas possuem de 0 a 2 campos com a tag H1.

Onde o classificados não possui uma tag h1 em seu corpo, enquanto o doctoralia e o telelistas possuem essa tag 2x em seu corpo, além disto o domínio <b>psicolink</b> possui 1 página outlier que possui 2 tags H1 na mesma página, enquanto todas as demais possuem apenas 1.

A maioria dos domínios pode se utilizar apenas pegar o primeiro campo H1 para se extrair o campo nome, porém os domínios Psicólgoos do Brasil, telelistas e classificados precisam de um tratamento extra para serem cobertos.

Segue abaixo suas particularidades.
O domínio <b>classificados</b> não possui a tag h1, onde é possível observar que seus campos estão sempre dentro de uma tabela, talvez esse domínio seja o mais diferente dentre os utilizados.

O <b>telelistas</b> é o único cuja tag <b>H1</b> esta dentro de uma tag <b>A</b> e esta deve ser ignorada, também sendo o único em que o campo nome não esta na primeira tag <b>H1</b> encontrada e ainda apesar do campo nome se encontrar na segunda tag <b>H1</b> o mesmo vem com a cidade pós-fixada dentro do próprio campo.

O domínio <b>psicólogos do brasil</b> possui o campo nome na tag <b>H1</b>, porém o mesmo trás como prefixo o nome do próprio domínio, enquanto o nome correto sem o domínio, se encontra em uma tag h2.

Assim com o algoritmo de extrair o campo diretamente da primeira tag <b>H1</b> encontrada, possuimos:

<b>Cobertura: 87%</b>
<b>Precisão: 71.8%</b>

### Melhorando a extração dos nomes
Desta forma a primeira estratégia que se vem a mente para aumentar a precisão do extrator genérico é adicionar condicionais a cerca da seleção da tag quando encontrarmos mais que uma tag <b>H1</b> na página, ao simplesmente alterar a estratégia para pegar a última tag <b>H1</b>, obtemos uma <b>Cobertura de 87% </b> e <b>Precisão de 84.5%</b> pois coletaremos corretamente todas as páginas do domínio <b>Telelistas</b>, porém o outlier encontrado no domínio <b>psicolink</b> não será extraido corretamente.

In [23]:
for index, soup in enumerate(soups):
    h1s = soup.find_all('h1')
    if(len(h1s) == 1):
        print(str(index) + '-' + clean_text(h1s[0].text))
    elif(len(h1s) == 2):
        print(str(index) + '-' + clean_text(h1s[1].text))
    else:
        print( str(index) + '-Empty result')

0-Empty result
1-Empty result
2-Empty result
3-Empty result
4-Empty result
5-Empty result
6-Empty result
7-Empty result
8-Empty result
9-Empty result
10-Sérgio Alexandre Alves Fernandes
11-Luiz Gustavo Canuto
12-Igor Costa
13-Bruno Schievenin Madi
14-Sandro Azevêdo
15-Wilza M Brosig
16-Dra. Luciana Dias Nascimento
17-Fernanda Simionato Costa
18-Rodrigo Noia Mattos Montan
19-Kassia Arouca cardoso
20-Dr. Caio Fasoli Rebouças
21-ADRIANA PANZAN
22-CONSULTORIO DE PSICOLOGIA SORAIA S GRANADO RODRIGUES
23-PSICOVIDA - PSICOLOGIA & NEUROPSICOLOGIA
24-SIMONE GUIMARAES PSICOLOGIA
25-ANA MEDIOLARO PSICOLOGIA CLINICA
26-DANIELLE CRISTINA GAZZONI
27-DR. ADRIANO MACHADO OLIVEIRA - SALUTE 21
28-LEILA  DOURADO
29-ESPAÇO COMPORTAMENTAL
30-ANDREA ZANETTI
31-Mônica Komora
32-Abdon Sardinha
33-Espaço Consciência Psicologia e outras Terapias
34-Sandra Pereira
35-Vera Pelizzari Psicóloga Clínica
36-Novos Horizontes
37-Ussénade Maria de Oliveira
38-Sumara Gonçalves
39-Cristyani Paiva
40-Lia Soares Dantas
41-H

Com um pequeno processamento extra para remover o que se encontra antes de um <b> - </b> podemos melhorar ainda mais a precisão.
Porém existe uma chance de isto ser considerado um overfitting e poder errar quando novos domínios forem adicionados. Com esta modificação o a precisão aumenta para <b> 98.5%</b>, manténdo os <b>84.5%</b> de cobertura

In [24]:
def clean_text(text):
    start = max(0,text.find('- '))
    if(start >0):
        start = start + 2
    text = text[start:]
    return text.replace('\n', '').replace('\t', '').strip();

for index, soup in enumerate(soups):
    h1s = soup.find_all('h1')
    if(len(h1s) == 1):
        print(str(index) + '-' + clean_text(h1s[0].text))
    elif(len(h1s) == 2):
        print(str(index) + '-' + clean_text(h1s[1].text))
    else:
        print( str(index) + '-Empty result')


0-Empty result
1-Empty result
2-Empty result
3-Empty result
4-Empty result
5-Empty result
6-Empty result
7-Empty result
8-Empty result
9-Empty result
10-Sérgio Alexandre Alves Fernandes
11-Luiz Gustavo Canuto
12-Igor Costa
13-Bruno Schievenin Madi
14-Sandro Azevêdo
15-Wilza M Brosig
16-Dra. Luciana Dias Nascimento
17-Fernanda Simionato Costa
18-Rodrigo Noia Mattos Montan
19-Kassia Arouca cardoso
20-Dr. Caio Fasoli Rebouças
21-ADRIANA PANZAN
22-CONSULTORIO DE PSICOLOGIA SORAIA S GRANADO RODRIGUES
23-PSICOLOGIA & NEUROPSICOLOGIA
24-SIMONE GUIMARAES PSICOLOGIA
25-ANA MEDIOLARO PSICOLOGIA CLINICA
26-DANIELLE CRISTINA GAZZONI
27-SALUTE 21
28-LEILA  DOURADO
29-ESPAÇO COMPORTAMENTAL
30-ANDREA ZANETTI
31-Mônica Komora
32-Abdon Sardinha
33-Espaço Consciência Psicologia e outras Terapias
34-Sandra Pereira
35-Vera Pelizzari Psicóloga Clínica
36-Novos Horizontes
37-Ussénade Maria de Oliveira
38-Sumara Gonçalves
39-Cristyani Paiva
40-Lia Soares Dantas
41-Hildes do Amparo Delduque Farina
42-Silvia R

In [25]:
names = []

for index, soup in enumerate(soups):
    h1s = soup.find_all('h1')
    if(len(h1s) == 1):
        names.append(clean_text(h1s[0].text))
    elif(len(h1s) == 2):
        names.append(clean_text(h1s[1].text))
    else:
        print( str(index) + '-Empty result')
        names.append('')

d = {'names': names}
df = pd.DataFrame(data=d)
df

0-Empty result
1-Empty result
2-Empty result
3-Empty result
4-Empty result
5-Empty result
6-Empty result
7-Empty result
8-Empty result
9-Empty result


,names
0,
1,
2,
3,
4,
5,
6,
7,
8,
9,


## Coletando telefones

In [90]:
phones = []
primary_phone = []

def clean_text(text):
    return text.replace('\n', '').replace('\t', '').strip().encode('ascii',errors='ignore').decode()

def remove_prefix(text):
    position = text.find(':')+1
    return text[position:].strip()

def is_phone(text):
    separator_position = text.find('-')
    result = text[0]=='(' or ( separator_position > 0 and len(text[separator_position:])>4 )
    result = result or len(text)>8
#    result = result or text.find(' ') > 0
    print(text)
    return result

def choice_result(texts):
    result = []
    for text in texts:
        cleaned_text = clean_text(text[0])
        cleaned_text = remove_prefix(cleaned_text)
        if(is_phone(cleaned_text)):
            result.append(cleaned_text)
    print(result)
    return ', '.join(list(set(result)))

#re.findall(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,4}",s)
regex = r"(\w[\w\d_]*:\s*(\(\d*\))?(\d*)?\s*\d{3,}-?\d+)"
rg = re.compile(regex)

for index, soup in enumerate(soups):
    for script in soup(["script", "style"]):
        script.decompose()    # rip it out
    test_str = soup.get_text()

    result = rg.findall(test_str)
    if result:
        print(str(index) + ' ' + str(len(result)) + str(result))
        result = choice_result(result)
        phones.append(result)
    else:
        phones.append('')


0 2[('Contato: \xa0\n11 996970779', '', '11'), ('Telefone:\n1128632313', '', '112863')]
11 996970779
1128632313
['11 996970779', '1128632313']
1 2[('Contato: \xa0\n(34)99880-6090', '(34)', '99'), ('Telefone:\n(34)99880-6090', '(34)', '99')]
(34)99880-6090
(34)99880-6090
['(34)99880-6090', '(34)99880-6090']
2 1[('Telefone:\n999386868', '', '99938')]
999386868
['999386868']
3 2[('Contato: \xa0\n981343194', '', '98134'), ('Telefone:\n981453194', '', '98145')]
981343194
981453194
['981343194', '981453194']
4 2[('Fones: 2545-1075', '', '2'), ('Telefone:\n2545-1075', '', '2')]
2545-1075
2545-1075
['2545-1075', '2545-1075']
5 1[('Telefone:\n21979402189', '', '2197940')]
21979402189
['21979402189']
6 1[('Telefone:\n(51)98004640', '(51)', '9800')]
(51)98004640
['(51)98004640']
7 2[('Contato: \xa0\n11 99567-5258', '', '11'), ('Telefone:\n11 99567-5258', '', '11')]
11 99567-5258
11 99567-5258
['11 99567-5258', '11 99567-5258']
8 2[('Contato: \xa0\n11 99567-5258', '', '11'), ('Telefone:\n11 99567-

In [91]:
phones_counter = 0
for index, phone in enumerate(phones):
    print(str(index) + ' + ' + phone)
    if phone:
        phones_counter = phones_counter + 1
print(phones_counter)
d = {'names':names, 'phones': phones}
df = pd.DataFrame(data=d)
df

0 + 11 996970779, 1128632313
1 + (34)99880-6090
2 + 999386868
3 + 981343194, 981453194
4 + 2545-1075
5 + 21979402189
6 + (51)98004640
7 + 11 99567-5258
8 + 11 99567-5258
9 + (48)34654498
10 + 
11 + 
12 + 
13 + 
14 + 
15 + 
16 + 
17 + (19) 98290-2433
18 + 
19 + 
20 + 
21 + 13201-839
22 + 09624-030
23 + 06010-170
24 + 09770-000
25 + 09770-000
26 + 79010-200
27 + 77020-494
28 + 66050-110
29 + 86015-010
30 + 05302-001
31 + 
32 + 
33 + 
34 + 
35 + 
36 + 
37 + 
38 + 
39 + 
40 + 
41 + 
42 + 
43 + 
44 + 
45 + 
46 + 
47 + 
48 + 
49 + 
50 + 
51 + (11) 2838-9818
52 + (11) 2838-9818
53 + (11) 2838-9818
54 + (11) 2838-9818
55 + (11) 2838-9818
56 + (11) 2838-9818
57 + (11) 2838-9818
58 + (11) 2838-9818
59 + (11) 2838-9818
60 + (11) 2838-9818
61 + (21) 99623-65, (21) 2205-78
62 + (21) 2267-06, (21) 2547-92, (21) 99131-47, (21) 2547-9287
63 + (81) 99975-78, (81) 3222-28
64 + (31) 3241-48
65 + (34) 3236-88
66 + (62) 99214-65, (62) 98595-68, (62) 98256-68
67 + (62) 98172-97
68 + (21) 2547-92, (21) 99363

,names,phones
0,,"11 996970779, 1128632313"
1,,(34)99880-6090
2,,999386868
3,,"981343194, 981453194"
4,,2545-1075
5,,21979402189
6,,(51)98004640
7,,11 99567-5258
8,,11 99567-5258
9,,(48)34654498


## analisando o resultado da coleta de telefones

Ao todo foram coletados telefone de 51 casos, representando uma <b>cobertura de 62,9%</b>

É possível observar alguns casos curiosos como no segundo domínio (doctoralia) que não possuí por default o número do psícologo, porém o algoritmo identificou 1 número na descrição do psícologo, ou nos domínios zenklub e psicologosdobrasil, onde todos psíclogos foram alocados para o mesmo número, pois os sites são cadastrado no conselho de psicologia e existe um responsável pelos sites, logo o CRP e telefone deste psicólogo responsável estão presente em todas as páginas causando uma falha na coleta.

Também foi possível observar que no domínio telelistas o algoritmo novamente se comportou de forma de forma diferente do algoritmo utilizado na extração específica, obtendo um resultado melhor que o mesmo, assim como no doctoralia, obtendo mais números de telefones do que estão presentes no campo específico, este resultado pode ser observado na páginad e Lena Lener.

A precisão obtida foi de <b>60%</b>.

A cobertura poderia ser ampliada afloxando mais as regras da coleta, como por exemplo: remover a regra de só coletar números que estejam após o caractere de dois pontos, porém isso faria coletar muitos telefones que não são do psicólogo em questão.



## Coletando CRPs

In [95]:
def clean_text(text):
    rg = re.compile(r"((\d{2}\/)?\d{5,6})")
    return rg.findall(text)[0][0]

CRPs = []

#re.findall(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,4}",s)
regex = r"((\W)(\d{2}\/)?\d{5,6}(\s))"
rg = re.compile(regex)

for index, soup in enumerate(soups):
    for script in soup(["script", "style"]):
        script.decompose()    # rip it out
    test_str = soup.get_text()

    result = rg.findall(test_str)
    #if index == 13:
     #   print(test_str)
    if result:
        print(str(index) + ' ' + str(len(result)) + str(result))
        result = str(result[0][0])
        CRPs.append(clean_text(result))
    else:
        CRPs.append('')

count_crps = 0
for crp in CRPs:
    if len(crp)>0:
        count_crps = count_crps + 1
count_crps

4 2[(' 46788 ', ' ', '', ' '), (' 46788\n', ' ', '', '\n')]
10 2[('-04/44975\n', '-', '04/', '\n'), ('-04/44975\n', '-', '04/', '\n')]
11 2[(' 36620\n', ' ', '', '\n'), (' 36620\n', ' ', '', '\n')]
12 4[(' 006853 ', ' ', '', ' '), (' 40150 ', ' ', '', ' '), (' 40150 ', ' ', '', ' '), (' 006853 ', ' ', '', ' ')]
14 2[(' 03/11004\n', ' ', '03/', '\n'), (' 03/11004\n', ' ', '03/', '\n')]
15 2[(' 08/18011\n', ' ', '08/', '\n'), (' 08/18011\n', ' ', '08/', '\n')]
17 2[(' 06/119704\n', ' ', '06/', '\n'), (' 06/119704\n', ' ', '06/', '\n')]
18 2[(' 06/105648\n', ' ', '06/', '\n'), (' 06/105648\n', ' ', '06/', '\n')]
19 2[(' 49917 ', ' ', '', ' '), (' 49917 ', ' ', '', ' ')]
20 2[(' 06/99296\n', ' ', '06/', '\n'), (' 06/99296\n', ' ', '06/', '\n')]
31 1[('\n06/118140\n', '\n', '06/', '\n')]
32 1[(' 01/17965\n', ' ', '01/', '\n')]
33 1[('-01/10631\n', '-', '01/', '\n')]
34 1[(' 04738\n', ' ', '', '\n')]
35 1[(' 06/120070\n', ' ', '06/', '\n')]
37 1[('\n02/17845\n', '\n', '02/', '\n')]
38 1[(' 0

37

In [96]:
d = {'names':names, 'phones': phones, 'CRPs': CRPs}
df = pd.DataFrame(data=d)
df = df[['names', 'phones', 'CRPs']]
df

,names,phones,CRPs
0,,"11 996970779, 1128632313",
1,,(34)99880-6090,
2,,999386868,
3,,"981343194, 981453194",
4,,2545-1075,46788
5,,21979402189,
6,,(51)98004640,
7,,11 99567-5258,
8,,11 99567-5258,
9,,(48)34654498,


### Analisando coleta dos CRPs

No primeiro domínio foi possível encontrar 1 CRP, ao observar a página foi possível perceber que se trata de um CRP que foi inserido na descrição do psicólogo, o que é explicável pelo fato de ser um site de anuncio mais genérico (classificados).

No segundo domínio foi possível coletar o CRP de 9 dos 11 psicólogos, onde os 2 casos errados foram correspondentes a 1 CRP no padrão errado e outro e outro com 4 digitos, suponho que este CRP esteja incompleto, os demais resultados nesse site se mostraram iguais ou melhores que o extrator específico, o mesmo ocorreu com 1 psicólogo do site mundopsicologos.

Por fim houve mais uma coleta estranha no penúltimo psicólogo, cujo CRP é 14/04457-7, único caso avistado onde se possui um - e um número após.

A precisão pode ser considerada 100% caso esses sejam considerados erros na inserção dos dados, ou 90% casos sejam considerados errados. 

A precisão encontrada foi 100%, acredita-se que por se tratar de um campo de formato bastante restrito e normalmente informado, principalmente em sites do domínio, a precisão realmente é bastante alta.

# Coletando preço

In [103]:
def clean_text(text):
    return text

prices = []

#re.findall(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,4}",s)
regex = r"((R\$)(\s*)(\d*))"
rg = re.compile(regex)

for index, soup in enumerate(soups):
    for script in soup(["script", "style"]):
        script.decompose()    # rip it out
    test_str = soup.get_text()

    result = rg.findall(test_str)
    if result:
        print(str(index) + ' ' + str(len(result)) + str(result))
        result = str(result[0][0])
        prices.append(clean_text(result))
    else:
        prices.append('')

count_prices = 0
for crp in prices:
    if len(crp)>0:
        count_prices = count_prices + 1
count_prices

0 1[('R$ 165', 'R$', ' ', '165')]
1 1[('R$ 0', 'R$', ' ', '0')]
2 1[('R$ 70', 'R$', ' ', '70')]
3 1[('R$ 0', 'R$', ' ', '0')]
4 1[('R$ 0', 'R$', ' ', '0')]
5 1[('R$ 0', 'R$', ' ', '0')]
6 1[('R$ 0', 'R$', ' ', '0')]
7 1[('R$ 0', 'R$', ' ', '0')]
8 1[('R$ 0', 'R$', ' ', '0')]
9 1[('R$ 0', 'R$', ' ', '0')]
10 5[('R$ 0', 'R$', ' ', '0'), ('R$ 150', 'R$', ' ', '150'), ('R$ 150', 'R$', ' ', '150'), ('R$ 150', 'R$', ' ', '150'), ('R$ 150', 'R$', ' ', '150')]
31 1[('R$ 100', 'R$', ' ', '100')]
32 1[('R$ 100', 'R$', ' ', '100')]
34 1[('R$ 100', 'R$', ' ', '100')]
35 1[('R$ 100', 'R$', ' ', '100')]
36 2[('R$ 60', 'R$', ' ', '60'), ('R$ 100', 'R$', ' ', '100')]
37 2[('R$ 60', 'R$', ' ', '60'), ('R$ 100', 'R$', ' ', '100')]
39 1[('R$ 100', 'R$', ' ', '100')]
40 1[('R$ 100', 'R$', ' ', '100')]
41 1[('R$105', 'R$', '', '105')]
42 1[('R$110', 'R$', '', '110')]
43 1[('R$170', 'R$', '', '170')]
44 1[('R$90', 'R$', '', '90')]
45 1[('R$150', 'R$', '', '150')]
46 1[('R$90', 'R$', '', '90')]
47 1[('R$150'

39

In [104]:
d = {'names':names, 'phones': phones, 'CRPs': CRPs, 'prices': prices}
df = pd.DataFrame(data=d)
df = df[['names', 'phones', 'CRPs', 'prices']]
df

,names,phones,CRPs,prices
0,,"11 996970779, 1128632313",,R$ 165
1,,(34)99880-6090,,R$ 0
2,,999386868,,R$ 70
3,,"981343194, 981453194",,R$ 0
4,,2545-1075,46788,R$ 0
5,,21979402189,,R$ 0
6,,(51)98004640,,R$ 0
7,,11 99567-5258,,R$ 0
8,,11 99567-5258,,R$ 0
9,,(48)34654498,,R$ 0


### Analisando coleta dos preços

Mais uma vez foi possível coletar todos os preços, porém aqui ficam algumas observações.
Alguns campos ficaram zerados pois alguns psicólogos não informam ou preço, deixando zero, ou oferecem uma primeira consulta de graça.

Algumas coletas foram realizadas a mais que no extrator específico pois dessa vez foi possível observar o preço que alguns psicólogos colocaram na descrição em sites que não possuem o campo de preço.

Alguns psicólogos exibiram mais de um preço, por ofertarem serviços diferentes e/ou premeira consulta free, nestes casos, foi considerado apenas o primeiro preço, porém é possível coletar todos, apesar de ser necessário um trabalho adicional para dar significado aos valores.